# Step 3: Tool Selection

## The Key Insight

**Agents use tools when needed, not reflexively.**

This is what separates **"agentic" systems** from **rigid workflows**.

### Workflow vs. Agent

| Workflow | Agent |
|----------|-------|
| Fixed sequence of steps | Dynamic decision-making |
| Always executes all tools | Uses tools only when needed |
| Predictable, rigid | Adaptive, flexible |
| Good for: Known processes | Good for: Varied user needs / complex tasks |

## What We'll Demonstrate

We'll give the agent **two tools**
- get_weather_for_city
- get_trending_news_for_city

and test with differet prompts.

## Setup

Same as Step 2, but now we're importing two different tool functions.

In [1]:
# Note: This (nest_asyncio) is only required in Jupyter notebooks (https://ai.pydantic.dev/troubleshooting/)
import nest_asyncio

nest_asyncio.apply()

In [2]:
from pydantic_ai import Agent, RunContext
from pydantic import BaseModel
from tools import get_weather_for_city, get_trending_news_for_city
import dotenv

dotenv.load_dotenv(override=True)

True

## Define User Context

Same user model as before.

In [3]:
class UserInfo(BaseModel):
    name: str
    city: str

## Create the Agent

Same agent setup as Step 2.

In [4]:
basic_agent = Agent(
    model="anthropic:claude-3-5-haiku-latest",
    deps_type=UserInfo,
)


@basic_agent.system_prompt
def set_system_prompt(ctx: RunContext[UserInfo]) -> str:
    return f"""
    You are a helpful assistant.
    Answer questions as best you can, using any tools as needed.
    The user's name is {ctx.deps.name} and they are in {ctx.deps.city}.
    """

## Register Two Tools

Now we'll give the agent **two different tools**. Both are available, but the agent will intelligently choose:
- Which one to use
- Whether to use any at all

**Key point:** The docstrings tell the agent what each tool does. Make them clear and specific!

In [5]:
@basic_agent.tool
def tool_get_weather(ctx: RunContext[UserInfo]) -> str:
    """Check today's weather in a given location"""
    print(">> TOOL USED: Getting weather")
    return get_weather_for_city(ctx.deps.city)

In [6]:
@basic_agent.tool
def tool_get_trending_news(ctx: RunContext[UserInfo]) -> str:
    """Get today's trending news item in a given location"""
    print(">> TOOL USED: Getting trending news")
    return get_trending_news_for_city(ctx.deps.city)

## Set Up Our Test User

We'll use the same user for all three tests to make comparison easier.

In [7]:
user_info = UserInfo(name="JP", city="Edinburgh")

## Test 1: Weather Question

What tool will the agent use?

In [8]:
prompt = "What's the weather like today where I am?"

print(f"{'=' * 60}")
print(f"TEST 1: Weather Question")
print(f"PROMPT: {prompt}")
print(f"{'=' * 60}\n")

response = basic_agent.run_sync(user_prompt=prompt, deps=user_info)

print(f"\nAgent response:")
print(response.output)
print()

TEST 1: Weather Question
PROMPT: What's the weather like today where I am?

>> TOOL USED: Getting weather

Agent response:
It looks like it's a typical Scottish day in Edinburgh - rainy and cool. The temperature is around 6°C, so you might want to grab a waterproof jacket and an umbrella if you're heading out today. It's a good day to stay warm and perhaps enjoy some indoor activities or a cozy café.

Is there anything else you'd like to know about today's weather or plans?



### Analysis

Did you see "TOOL USED: Getting weather"? ✅

The agent:
1. Read the question
2. Identified it needs current weather data
3. Chose the weather tool (not the news tool!)
4. Used the result to answer

## Test 2: News Question

What tool will the agent use?

Note this is the same user and the same agent. The only difference is the question.

In [9]:
prompt = "What's the trending news item where I am today?"

print(f"{'=' * 60}")
print(f"TEST 2: News Question")
print(f"PROMPT: {prompt}")
print(f"{'=' * 60}\n")

response = basic_agent.run_sync(user_prompt=prompt, deps=user_info)

print(f"\nAgent response:")
print(response.output)
print()

TEST 2: News Question
PROMPT: What's the trending news item where I am today?

>> TOOL USED: Getting trending news

Agent response:
Well, that's an unusual piece of news! According to the trending news tool, today in Edinburgh there's a story about a surprisingly large mongoose that was seen chasing a startled park manager. It sounds like quite an unexpected and amusing incident. Would you like to hear more about this or discuss anything else?



### Analysis

Did you see "TOOL USED: Getting trending news"? ✅

The agent:
1. Read the question
2. Identified it needs trending news
3. Chose the NEWS tool (not weather!)
4. Ignored the weather tool completely

**This is smart tool selection!**

## Test 3: Geography Question

What about this question?

In [10]:
prompt = "What country is my city in? Has it always been the case?"

print(f"{'=' * 60}")
print(f"TEST 3: Geography Question")
print(f"PROMPT: {prompt}")
print(f"{'=' * 60}\n")

response = basic_agent.run_sync(user_prompt=prompt, deps=user_info)

print(f"\nAgent response:")
print(response.output)
print()

TEST 3: Geography Question
PROMPT: What country is my city in? Has it always been the case?


Agent response:
Edinburgh is a city located in Scotland, which is part of the United Kingdom. 

Historically, Scotland was an independent kingdom for many centuries before the Acts of Union in 1707, which united the Kingdom of Scotland and the Kingdom of England to form the Kingdom of Great Britain. So no, it has not always been part of the United Kingdom as we know it today.

Before 1707, Scotland was a separate sovereign state with its own monarchy, government, and international relations. The union with England was primarily driven by economic and political considerations, including Scotland's financial difficulties after the unsuccessful Darien Scheme (a failed colonial expedition) and the desire for a stable succession to the throne.

Today, Scotland is one of the four constituent countries of the United Kingdom, along with England, Wales, and Northern Ireland. It has a significant degree

### Analysis

Did you see any "TOOL USED" messages? (You should not see any 😉)

The agent:
1. Read the question
2. Realized it's about geography/history
3. Checked its own knowledge
4. Answered directly WITHOUT calling any tools!

**This is the key insight:** Good agents don't just use tools reflexively. They use them **when needed**.

## Key Takeaways

### Agentic Behavior = Conditional Tool Use

| System Type | Tool Usage |
|-------------|------------|
| **Workflow** | Always calls all tools in fixed sequence |
| **Agent** | Decides which tools (if any) based on the question |

### Why This Matters

✅ **LLM / Agents can replace complex logic on what tool / function to call**

### How the Agent Decides

The LLM looks at:
1. **The user's question** - What information do they need?
2. **Available tools** - What can I do? (reads docstrings)
3. **Its own knowledge** - Do I already know this?
4. **Decision** - Use tool X, tool Y, or no tool at all

This reasoning happens **automatically** - you just define the tools and let the agent decide!

## When to Use Workflows vs. Agents

### Use Workflows When:
- ✅ Steps are always required (e.g., data validation pipeline)
- ✅ Input and output are well-defined
- ✅ Example: Processing payment transactions

### Use Agents When:
- ✅ User intent varies widely
- ✅ Tools should be used conditionally
- ✅ Natural language interaction
- ✅ Example: Customer support chatbot, research assistant

**Sometimes you need both!** An agent might trigger a workflow for specific tasks.

## What's Next?

Steps 5-6 show more complex examples, using data stored in a Weaviate database. Here are the tools we'll use:

In [12]:
from tools import search_weaviate_docs, fetch_weaviate_docs_page

query = "collection aliases in Weaviate"

r = search_weaviate_docs(query)

print("\nSummary of search results:")
for i in r:
    print(i["path"])
    print(i["summary"])

Returned results:
weaviate/manage-collections/collection-aliases.mdx
weaviate/tutorials/collection-aliases.mdx
weaviate/manage-collections/collection-operations.mdx
weaviate/config-refs/collections.mdx
weaviate/starter-guides/managing-collections/index.mdx

Summary of search results:
weaviate/manage-collections/collection-aliases.mdx
A guide to creating, managing, and using collection aliases in Weaviate, which allow alternative names for collections and enable zero-downtime migrations.
weaviate/tutorials/collection-aliases.mdx
A comprehensive tutorial demonstrating how to use Weaviate collection aliases to perform zero-downtime migrations, enabling seamless schema updates and collection version management without service interruption.
weaviate/manage-collections/collection-operations.mdx
This document provides a comprehensive guide to managing Weaviate collections, including creating, reading, updating, and deleting collections with code examples across multiple programming languages.

In [13]:
print("\nFull page contents:")
for i in r:
    page = fetch_weaviate_docs_page(i["path"])
    print("Page length: ", len(page), "characters")
    print(page[:100] + "...")
    print("\n")


Full page contents:
Page length:  17558 characters
---
title: Collection aliases
sidebar_label: Collection aliases
image: og/docs/howto.jpg
---

import...


Page length:  21582 characters
---
title: Zero-downtime collection migration with aliases
description: Learn how to migrate Weaviat...


Page length:  52841 characters
---
title: Basic collection operations
sidebar_label: Collection operations (CRUD)
image: og/docs/ho...


Page length:  91745 characters
---
title: Collection definition
description: Reference for Weaviate collection parameters.
---

imp...


Page length:  36490 characters
---
title: Collection definitions (schemas)
description: Begin with collection definition setup in W...




---

**⏸️ PAUSE: Questions before we move to Weaviate integration?**